In [0]:
-- Create widgets for catalog and schema
CREATE WIDGET TEXT catalog DEFAULT '';
CREATE WIDGET TEXT schema DEFAULT '';

In [0]:
SELECT ai_parse_document(content) AS parsed_document
  FROM READ_FILES(
    CONCAT('/Volumes/', :catalog, '/', :schema, '/raw_data/pdf_documentation/'),
    format => 'binaryFile'
  ) 
  LIMIT 1;

In [0]:
USE CATALOG IDENTIFIER(:catalog);
USE SCHEMA IDENTIFIER(:schema);

In [0]:
CREATE OR REPLACE TABLE parsed_policy_pdfs (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  product_name STRING,
  title STRING,
  content STRING,
  doc_uri STRING)
  TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
SELECT array_join(transform(CAST(parsed_document:document:elements as ARRAY<VARIANT>), x -> x:content), '\n') as content,
       path as doc_uri
FROM (
  SELECT ai_parse_document(content) AS parsed_document, path
  FROM READ_FILES('/Volumes/hytech_workshop/ai_agent/raw_data/pdf_documentation/', format => 'binaryFile') limit 1
)

In [0]:
%sql
INSERT INTO TABLE parsed_policy_pdfs (product_name, title, content, doc_uri)
SELECT ai_extract.product_name, ai_extract.title, content, doc_uri
FROM (
  SELECT
    ai_extract(content, array('product_name', 'title')) AS ai_extract,
    content,
    doc_uri
  FROM (
    SELECT array_join(transform(CAST(parsed_document:document:elements as ARRAY<VARIANT>), x -> x:content), '\n') as content,
          path as doc_uri
    FROM (
      SELECT ai_parse_document(content) AS parsed_document, path
      FROM READ_FILES(
        CONCAT('/Volumes/', :catalog, '/', :schema, '/raw_data/pdf_documentation/'),
        format => 'binaryFile'
      ) LIMIT 5 -- We are limiting this to 5 for this workshop.
    )
  )
);

In [0]:
select * from parsed_policy_pdfs;